# Testing

In this notebook you can explore and test the resulting database tables from our Dagster ETL process.
Here you can analyze


In [9]:
import duckdb
import polars as pl

In [12]:
%load_ext sql
conn = duckdb.connect(database="../dsp-dagster/storage/DSP.db")
%sql conn --alias duckdb
%sql SHOW ALL TABLES;

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Running query in 'duckdb'

database,schema,name,column_names,column_types,temporary


In [11]:
cbs_wijken = conn.execute(
    """
    SELECT * FROM public.cbs_wijken """
).pl()

service_areas = conn.execute(
    """
    SELECT * FROM public.service_areas """
).pl()

deployment_incident_vehicles_weather = conn.execute(
    """
    SELECT * FROM joined.deployment_incident_vehicles_weather"""
).pl()

deployment_incident_vehicles = conn.execute(
    """
    SELECT * FROM joined.deployment_incident_vehicles"""
).pl()


adjust_knmi_data_types = conn.execute(
    """
    SELECT * FROM cleaned.adjust_knmi_data_types
    """
).pl()


# Close the database connection
conn.close()

CatalogException: Catalog Error: Table with name cbs_wijken does not exist!
Did you mean "temp.pg_catalog.pg_enum"?
LINE 2:     SELECT * FROM public.cbs_wijken 
                          ^

In [8]:
# cbs_wijken.head(10)

In [5]:
adjust_knmi_data_types.head()

station_code,date,hour,DD,FH,FF,FX,T,T10N,TD,SQ,Q,DR,RH,P,VV,N,U,WW,IX,M,R,S,O,Y
i64,date,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
240,2005-01-01,1,260,40,30,60,68,null,57,0,0,0,0,10246,57,8,93,10,7,0,0,0,0,0
240,2005-01-01,2,230,30,30,60,65,null,52,0,0,0,0,10244,58,8,91,10,7,0,0,0,0,0
240,2005-01-01,3,230,40,30,50,43,null,34,0,0,0,0,10241,40,1,94,10,7,0,0,0,0,0
240,2005-01-01,4,220,40,40,50,38,null,32,0,0,0,0,10239,12,0,96,10,7,0,0,0,0,0
240,2005-01-01,5,230,40,40,50,38,null,34,0,0,0,0,10237,14,3,97,10,7,0,0,0,0,0


In [6]:
# display(service_areas)

H_Verzorgingsgebied_ID,Verzorgingsgebied,LAT,LON,geometry
i64,str,f64,f64,str
60,"""Aalsmeer""",52.262846,4.76409,"""POLYGON ((1103…"
62,"""Amstelveen""",52.306531,4.888029,"""MULTIPOLYGON (…"
64,"""Anton""",52.308747,4.980264,"""MULTIPOLYGON (…"
70,"""Diemen""",52.341314,4.964931,"""MULTIPOLYGON (…"
71,"""Dirk""",52.360768,4.886359,"""MULTIPOLYGON (…"
72,"""Driemond""",52.316452,5.020421,"""MULTIPOLYGON (…"
73,"""Duivendrecht""",52.333488,4.944756,"""MULTIPOLYGON (…"
74,"""GBA""",52.399355,4.798441,"""MULTIPOLYGON (…"
79,"""Hendrik""",52.375786,4.875513,"""POLYGON ((1192…"


In [14]:
deployment_incident_vehicles.head()

Incident_ID,Date,Incident_Starttime,Incident_Endtime,Incident_Duration,Incident_Priority,Service_Area,Municipality,Damage_Type,LON,LAT,Incident_Starttime_Hour,Incident_Endtime_Hour,Incident_Duration_Hour,Incident_Starttime_Minute,Incident_Endtime_Minute,Incident_Duration_Minute,Deployment_ID,Vehicle_Type,Vehicle_Role,Fire_Station,Fire_Station_Service_Status,Driving_Time_To_Incident,Vehicle
i64,date,time,time,time,i64,str,str,str,f64,f64,i8,i8,i8,i8,i8,i8,i64,str,str,str,str,str,str
511,2018-09-07,08:25:18,10:00:56,01:35:38,2,"""Amstelveen""","""Amstelveen""","""Tree""",4.838685,52.281552,8,10,1,25,0,35,1,"""TS""","""TS""","""Amstelveen""","""Professional""","""708""","""Tankautospuit"""
511,2018-09-07,08:25:18,10:00:56,01:35:38,2,"""Amstelveen""","""Amstelveen""","""Tree""",4.838685,52.281552,8,10,1,25,0,35,2,"""AL""","""RV""","""Amstelveen""","""Professional""","""786""","""Autoladder"""
738,2018-09-10,16:46:38,18:00:55,01:14:17,2,"""Victor""","""Amsterdam""","""Tree""",4.930968,52.359724,16,18,1,46,0,14,3,"""TS""","""TS""","""Victor""","""Professional""","""116""","""Tankautospuit"""
738,2018-09-10,16:46:38,18:00:55,01:14:17,2,"""Victor""","""Amsterdam""","""Tree""",4.930968,52.359724,16,18,1,46,0,14,4,"""AL""","""RV""","""Victor""","""Professional""","""114""","""Autoladder"""
739,2018-09-10,17:11:20,18:01:00,00:49:40,2,"""Dirk""","""Amsterdam""","""Building""",4.898818,52.357071,17,18,0,11,1,49,5,"""AL""","""RV""","""Dirk""","""Professional""","""438""","""Autoladder"""


In [7]:
display(deployment_incident_vehicles_weather.head(10))

Station_code,Date,Hour,Dd,Fh,Ff,Fx,T,T10n,Td,Sq,Q,Dr,Rh,P,Vv,N,U,Ww,Ix,M,R,S,O,Y,Incident_ID,Incident_Starttime,Incident_Endtime,Incident_Duration,Incident_Priority,Service_Area,Municipality,Damage_Type,LON,LAT,Incident_Starttime_Hour,Incident_Endtime_Hour,Incident_Duration_Hour,Incident_Starttime_Minute,Incident_Endtime_Minute,Incident_Duration_Minute,Deployment_ID,Vehicle_Type,Vehicle_Role,Fire_Station,Fire_Station_Service_Status,Driving_Time_To_Incident,Vehicle
i64,date,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,time,time,time,i64,str,str,str,f64,f64,i8,i8,i8,i8,i8,i8,i64,str,str,str,str,str,str
240,2005-01-01,1,260,40,30,60,68,null,57,0,0,0,0,10246,57,8,93,10,7,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
240,2005-01-01,2,230,30,30,60,65,null,52,0,0,0,0,10244,58,8,91,10,7,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
240,2005-01-01,3,230,40,30,50,43,null,34,0,0,0,0,10241,40,1,94,10,7,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
240,2005-01-01,4,220,40,40,50,38,null,32,0,0,0,0,10239,12,0,96,10,7,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
240,2005-01-01,5,230,40,40,50,38,null,34,0,0,0,0,10237,14,3,97,10,7,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
240,2005-01-01,6,200,40,40,50,32,21,29,0,0,0,0,10235,25,0,98,10,7,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
240,2005-01-01,7,200,40,50,60,40,null,38,0,0,0,0,10230,14,3,99,20,7,1,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
240,2005-01-01,8,200,50,50,70,49,null,46,0,0,0,0,10231,35,8,98,10,7,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
240,2005-01-01,9,200,50,50,70,55,null,51,0,5,0,0,10229,41,8,97,10,7,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
